<a href="https://colab.research.google.com/github/PhoebeLu1011/1132code/blob/main/WEEK8_%E6%96%87%E5%AD%97%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1chxnWQlMrqnnyQSzjaCT_hHOQfqah4hDEAsu4qOjTqI/edit?usp=sharing').sheet1

In [ ]:
dicts = gc.open_by_url('https://docs.google.com/spreadsheets/d/1chxnWQlMrqnnyQSzjaCT_hHOQfqah4hDEAsu4qOjTqI/edit?usp=sharing').get_worksheet(0)
dicts = dicts.get_all_records()
dicts = pd.DataFrame(dicts)

In [ ]:
# 讀取所有數據
rows = gsheets.get_all_records()

import pandas as pd
df = pd.DataFrame(rows)

In [ ]:
df.head()

,Student_ID,Content
0,S001,課程進度偏快，希望可以放慢一點。
1,S002,教材內容豐富，但部分章節太難。
2,S003,希望未來能有更多案例分享。
3,S004,希望作業能有更明確的說明。
4,S005,希望未來能有更多案例分享。


In [ ]:
df.describe()

,Student_ID,Content
count,50,50
unique,50,20
top,S001,上課氣氛活潑，讓人期待每一堂課。
freq,1,5


In [ ]:
!pip install jieba
!pip install snownlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 17.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for snownlp: filename=snownlp-0.12.3-py3-none-any.whl size=37760946 sha256=b8ee213e801ea7a46c64f10645ede79518a2e565fbc7e2098388dbd60e97e7f6
  Stored in directory: /root/.cache/pip/wheels/4a/fc/04/d1937c02b2a445b34754da55f71612a3df648a38d711bd17eb
Successfully built snownlp


In [ ]:
import jieba
from snownlp import SnowNLP
import pandas as pd
from collections import Counter
import re
import plotly.express as px

In [ ]:
for _, row in dicts.iterrows():
    word = row['Content']
    jieba.add_word(word, freq=10000)
    jieba.suggest_freq(word, tune=True)

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.507 seconds.
DEBUG:jieba:Loading model cost 1.507 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [ ]:
# 只保留中文字
def extract_chinese(text):
    return ''.join(re.findall(r'[\u4e00-\u9fa5]', text))

In [ ]:
df.head()

,Student_ID,Content
0,S001,課程進度偏快，希望可以放慢一點。
1,S002,教材內容豐富，但部分章節太難。
2,S003,希望未來能有更多案例分享。
3,S004,希望作業能有更明確的說明。
4,S005,希望未來能有更多案例分享。


In [ ]:
replace_dict = {
    "太難": "具有挑戰性",
    "不太清楚": "說明不夠明確",
    "收穫良多": "收穫很多",
    "可以放慢一點": "進度太快"
}

In [ ]:
def replace_words(text):
    for old_word, new_word in replace_dict.items():
        text = text.replace(old_word, new_word)
    return text

In [ ]:
df['Updated_Content'] = df['Content'].apply(replace_words)

In [ ]:
def clean_text(text):
    # 移除特殊符號
    text = re.sub(r'[!@#$%^&*()_+\-=\[\]{};\'\\:"|.<>/?~><？！~~...]', '', text)
    # 移除多餘空格
    text = re.sub(r'\s+', '', text)
    return text

In [ ]:
df['Cleaned_Content'] = df['Updated_Content'].apply(clean_text)

# 顯示處理後的資料
df[['Content', 'Updated_Content', 'Cleaned_Content']].head()

# 中文斷詞
df['Tokenized'] = df['Cleaned_Content'].apply(lambda x: list(jieba.cut(x, HMM=True)))
#df['Tokenized'] = df['Cleaned_Content'].apply(lambda x: list(jieba.cut(x, HMM=False)))

# 展平成所有詞語的列表
all_words = [word for tokens in df['Tokenized'] for word in tokens if len(word) > 1]


<ipython-input-15-b31a023f16df>:3: FutureWarning: Possible set symmetric difference at position 40
  text = re.sub(r'[!@#$%^&*()_+\-=\[\]{};\'\\:"|.<>/?~><？！~~...]', '', text)


In [ ]:
df['Cleaned_Content']

,Cleaned_Content
0,課程進度偏快，希望進度太快。
1,教材內容豐富，但部分章節具有挑戰性。
2,希望未來能有更多案例分享。
3,希望作業能有更明確的說明。
4,希望未來能有更多案例分享。
5,希望作業能有更明確的說明。
6,作業有挑戰性，學習很多實用技能。
7,希望可以錄影上課，方便複習。
8,課程進度偏快，希望進度太快。
9,老師講解清楚，讓我更容易理解內容。


In [ ]:
df['Tokenized']

,Tokenized
0,"[課程, 進度, 偏快, ，, 希望, 進度, 太快, 。]"
1,"[教材, 內容, 豐富, ，, 但, 部分, 章節, 具有, 挑戰性, 。]"
2,"[希望, 未來, 能, 有, 更, 多, 案例, 分享, 。]"
3,"[希望, 作業, 能, 有, 更, 明確, 的, 說明, 。]"
4,"[希望, 未來, 能, 有, 更, 多, 案例, 分享, 。]"
5,"[希望, 作業, 能, 有, 更, 明確, 的, 說明, 。]"
6,"[作業, 有, 挑戰性, ，, 學習, 很多, 實用, 技能, 。]"
7,"[希望, 可以, 錄影, 上, 課, ，, 方便, 複, 習, 。]"
8,"[課程, 進度, 偏快, ，, 希望, 進度, 太快, 。]"
9,"[老師, 講解, 清楚, ，, 讓, 我, 更, 容易, 理解, 內容, 。]"


In [ ]:
word_counts = Counter(all_words)
word_counts

Counter({'課程': 5,
         '進度': 4,
         '偏快': 2,
         '希望': 17,
         '太快': 2,
         '教材': 5,
         '內容': 7,
         '豐富': 2,
         '部分': 6,
         '章節': 2,
         '具有': 2,
         '挑戰性': 5,
         '未來': 4,
         '案例': 4,
         '分享': 4,
         '作業': 6,
         '明確': 5,
         '說明': 3,
         '學習': 5,
         '很多': 6,
         '實用': 3,
         '技能': 3,
         '可以': 5,
         '錄影': 2,
         '方便': 2,
         '老師': 10,
         '講解': 4,
         '清楚': 5,
         '容易': 8,
         '理解': 4,
         '小組': 2,
         '討論': 2,
         '幫助': 2,
         '效果': 2,
         '感覺': 2,
         '教學': 2,
         '用心': 2,
         '謝謝': 2,
         '耐心': 2,
         '願意': 2,
         '回答': 2,
         '問題': 2,
         '增加': 3,
         '實作': 3,
         '比例': 3,
         '安排': 3,
         '邏輯性': 3,
         '課氣氛': 5,
         '活潑': 5,
         '期待': 5,
         '一堂': 5,
         '課堂': 1,
         '互動': 1,
         '分心': 1,
         '整體': 2,
     

In [ ]:
# 排除停用詞（這裡提供一個簡單的停用詞列表，可根據需要擴展）
stop_words = set(['所以', '好','一堂', '因為', '大家', '的', '是', '了', '我', '也', '在', '和', '就','可以' , '不', '有', '他', '她', '你', '我們', '這個','希望'])
filtered_word_counts = {word: count for word, count in word_counts.items() if word not in stop_words}

In [ ]:
# 將詞頻轉為 DataFrame 並排序
word_freq_df = pd.DataFrame(filtered_word_counts.items(), columns=['Word', 'Frequency'])
word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

# 繪製關鍵字統計的條形圖
fig = px.bar(word_freq_df.head(30), x='Word', y='Frequency',
             title='關鍵字出現次數統計（前20名）',
             labels={'Word': '關鍵字', 'Frequency': '出現次數'},
             text='Frequency')

fig.update_layout(
    xaxis_title="關鍵字",
    yaxis_title="出現次數",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

In [ ]:
word_freq_df

,Word,Frequency
0,老師,10
1,容易,8
2,內容,7
3,部分,6
4,作業,6
...,...,...
67,落差,1
68,知識,1
69,講義,1
70,排版,1


In [ ]:
!apt-get update -qq
!apt-get install -y fonts-wqy-zenhei

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-wqy-zenhei
0 upgraded, 1 newly installed, 0 to remove and 47 not upgraded.
Need to get 7,472 kB of archives.
After this operation, 16.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-wqy-zenhei all 0.9.45-8 [7,472 kB]
Fetched 7,472 kB in 1s (5,663 kB/s)
Selecting previously unselected package fonts-wqy-zenhei.
(Reading database ... 126315 files and directories currently installed.)
Preparing to unpack .../fonts-wqy-zenhei_0.9.45-8_all.deb ...
Unpacking fonts-wqy-zenhei (0.9.45-8) ...
Setting up fonts-wqy-zenhei (0.9.45-8) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [ ]:
from wordcloud import WordCloud

# 取前 300 個最常出現的詞
top_words = word_freq_df.head(300)

# 使用 WQY Zenhei 字體
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    font_path='/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc'
)

# 根據詞頻生成文字雲
wordcloud.generate_from_frequencies(dict(zip(top_words['Word'], top_words['Frequency'])))

# 將文字雲轉換為圖像
wordcloud_img = wordcloud.to_array()

# 使用 Plotly 顯示文字雲
fig = px.imshow(wordcloud_img)
fig.update_layout(
    title="文字雲",
    xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    coloraxis_showscale=False
)
fig.show()